In [1]:
import time
from pathlib import Path
import pandas as pd
import numpy as np

import optuna as opt
from optuna.samplers import TPESampler
# suppress info logs
opt.logging.set_verbosity(opt.logging.WARNING)

from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor

/home/monsoon/miniconda3/envs/CS760/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extractArrays(df):
    '''Extracts feature matrix X and label array y from dataframe.'''
    return df.drop(['r_useful', 'r_id'], axis=1).values, df['r_useful'].values

Settings, load dataset, and constants

In [3]:
RANDOM_SEED = 760
DATA_DIR = Path("../../ready_data")

N_OPTUNA_TRIALS = 50
N_FOLDS = 5
N_REPS = 6 # number of repetitions of CV
T_ES = 20 # threshold # consecutive non-improvement rounds for early stopping

df_train = pd.read_parquet(DATA_DIR/"100K35F_train_main.parquet.snappy")
df_val = pd.read_parquet(DATA_DIR/"100K35F_val_main.parquet.snappy")
df_test = pd.read_parquet(DATA_DIR/"100K35F_test_main.parquet.snappy")

X_train, y_train = extractArrays(df_train)
X_val, y_val = extractArrays(df_val)
X_test, y_test = extractArrays(df_test)

print(f"Shape of the training data : {X_train.shape}")
print(f"Shape of the val data : {X_val.shape}")
print(f"Shape of the test data : {X_test.shape}")

Shape of the training data : (80000, 35)
Shape of the val data : (10000, 35)
Shape of the test data : (10000, 35)


Define model

In [4]:
MODEL_PREFIX = "xgb"
XGB_model = XGBRegressor(
    booster="gbtree",
    n_jobs=-1, # use all CPUs.
    tree_method="gpu_hist", # use GPU
    predictor="gpu_predictor",
    objective="reg:squarederror",
    eval_metric=["rmse"],
    random_state=RANDOM_SEED
)

# Mean imputation and standardisation
model_pipe = Pipeline([
    ("imp", SimpleImputer()),
    ("ss", StandardScaler()),
    (MODEL_PREFIX, XGB_model)])

# needed for setting parameters correctly in pipe
def hp_appender(hp_dict):
    '''Return dictionary where every key has the MODEL_PREFIX__ appended.'''
    new_dict = {}
    for key, val in hp_dict.items():
        new_dict[MODEL_PREFIX + "__" + key] = val
    return new_dict

Implement experiment procedure

In [5]:
def optuna_objective(trial, model, X, y):
    print(f"{time.strftime('%H:%M:%S', time.localtime())} | Running Optuna Trial: {trial.number}")
    
    # sample hyperparameters from optuna
    hyperparams = {
        "n_estimators":trial.suggest_int('n_estimators', 1, 501, step=5),
        "learning_rate" : trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
        "max_depth" : trial.suggest_int("max_depth", 2, 20),
        # subsample of observations for each iteration
        "subsample": trial.suggest_float("subsample", 0.5, 1, step=0.1),
        # subsample of features for each iteration
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1, step=0.1),
        "gamma": trial.suggest_float("gamma", 0, 1),
    }
    print(f"with hyperparameters: {hyperparams}")
    hyperparams = hp_appender(hyperparams)
    model.set_params(**hyperparams)

    # Inner CV Loop
    avg_score = -cross_val_score(model, X, y,
        scoring="neg_root_mean_squared_error", cv=KFold(N_FOLDS)).mean()
    print(f"complete! average cv RMSE: {avg_score}")
    return avg_score

In [6]:
class early_stopping_check_callback:
    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, study, trial):
        # stop study if the number of consecutive trials with no improvement is
        # at least the threshold.
        if trial.number - study.best_trial.number >= self.threshold:
            print("==== EARLY STOPPING ACTIVATED ====")
            study.stop()

In [7]:
def get_best_model(X, y):
    '''Obtains best hyperparameters using TPE sampling and cross-validation.'''
    study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
    study.optimize(
        lambda trial: optuna_objective(trial, model_pipe, X, y),
        n_trials=N_OPTUNA_TRIALS,
        callbacks=[early_stopping_check_callback(T_ES)]) # early stopping
    return study.best_params

In [8]:
def fit_and_score(model, hps, X_train, y_train, X_test, y_test):
    model.set_params(**hp_appender(hps))
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    # calculate scores
    rmse = mean_squared_error(y_test, y_preds, squared=False)
    mae = mean_absolute_error(y_test, y_preds)
    return rmse, mae

In [9]:
# actual repeated CV loop

repetition_results = {
    "hp": [],
    "rmse": [],
    "mae": []
}

for rep_i in range(N_REPS):
    print(f"====== Running repetition: {rep_i} ======")
    # shuffle training data. Make sure each iteration uses different seed
    cv_train = df_train.sample(frac=1, random_state=RANDOM_SEED+rep_i)
    # extract X and y arrays
    cv_X_train, cv_y_train = extractArrays(cv_train)
    
    # obtain best hyperparameters via cross-validation and TPE sampling.
    best_params = get_best_model(cv_X_train, cv_y_train)
    
    rmse, mae = fit_and_score(
        model_pipe, best_params,
        cv_X_train, cv_y_train,
        X_val, y_val)

    # save results for this iteration
    repetition_results["rmse"].append(rmse)
    repetition_results["mae"].append(mae)
    repetition_results["hp"].append(best_params)

====== Running repetition: 0 ======
13:51:37 | Running Optuna Trial: 0
with hyperparameters: {'n_estimators': 61, 'learning_rate': 0.0021542480108802785, 'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.7, 'gamma': 0.8606400505757477}
complete! average cv RMSE: 4.514703680199566
13:51:39 | Running Optuna Trial: 1
with hyperparameters: {'n_estimators': 81, 'learning_rate': 0.3336605688950222, 'max_depth': 14, 'subsample': 1.0, 'colsample_bytree': 0.5, 'gamma': 0.6755211563859722}
complete! average cv RMSE: 3.582254028226572
13:51:58 | Running Optuna Trial: 2
with hyperparameters: {'n_estimators': 221, 'learning_rate': 0.006094075338394031, 'max_depth': 20, 'subsample': 0.5, 'colsample_bytree': 0.7, 'gamma': 0.6522814819365693}
complete! average cv RMSE: 3.5548156960400767
13:58:07 | Running Optuna Trial: 3
with hyperparameters: {'n_estimators': 141, 'learning_rate': 0.07136913218405677, 'max_depth': 11, 'subsample': 0.8, 'colsample_bytree': 0.5, 'gamma': 0.9369507683216501}
compl

Obtain best overall model results

In [10]:
# combine train and val sets
# https://stackoverflow.com/questions/33356442/when-should-i-use-hstack-vstack-vs-append-vs-concatenate-vs-column-stack
X_train_val = np.vstack((X_train, X_val))
y_train_val = np.hstack((y_train, y_val))
print(X_train_val.shape)
print(y_train_val.shape)

(90000, 35)
(90000,)


In [13]:
rmse = repetition_results['rmse']
best_best_i = np.argmin(rmse) # best = minimum RMSE
hp = repetition_results['hp']
best_best_hp = hp[best_best_i]

rmse, mae = fit_and_score(model_pipe, best_best_hp,
    X_train_val, y_train_val, X_test, y_test)

print(f"Best overall HP:{best_best_hp}")
print(f"Best overall RMSE: {rmse}")
print(f"Best overall MAE: {mae}")

Best overall HP:{'n_estimators': 401, 'learning_rate': 0.022600313054882564, 'max_depth': 12, 'subsample': 0.7, 'colsample_bytree': 0.9, 'gamma': 0.06632386259883208}
Best overall RMSE: 3.0906995594281392
Best overall MAE: 1.327068233421445


Obtain mean and stdev statistics across the repetitions

In [16]:
print(repetition_results)

print("\nRMSE")
rmse = repetition_results['rmse']
print(f"mean: {np.mean(rmse)}, stdev: {np.std(rmse)}")

print("MAE")
mae = repetition_results['mae']
print(f"mean: {np.mean(mae)}, stdev: {np.std(mae)}")

{'hp': [{'n_estimators': 421, 'learning_rate': 0.021878698780591236, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.9, 'gamma': 0.0980062213505412}, {'n_estimators': 401, 'learning_rate': 0.022600313054882564, 'max_depth': 12, 'subsample': 0.7, 'colsample_bytree': 0.9, 'gamma': 0.06632386259883208}, {'n_estimators': 256, 'learning_rate': 0.026102504641773067, 'max_depth': 14, 'subsample': 0.7, 'colsample_bytree': 0.9, 'gamma': 0.09207759406403443}, {'n_estimators': 361, 'learning_rate': 0.025371908094027687, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.9, 'gamma': 0.7898155660784915}, {'n_estimators': 121, 'learning_rate': 0.0785657267086009, 'max_depth': 7, 'subsample': 0.5, 'colsample_bytree': 0.9, 'gamma': 0.9968205529982901}, {'n_estimators': 486, 'learning_rate': 0.019294993850654762, 'max_depth': 9, 'subsample': 0.5, 'colsample_bytree': 1.0, 'gamma': 0.39592777218157926}], 'rmse': [3.214096976147171, 3.1435160045767243, 3.2087057432716133, 3.1441555097398246,